In [ ]:
#| default_exp memory_training

# Training and Optimizing the Reflection Memory System

> A guide to training and saving the Reflection Memory component for Cogitarelink DSPy agents.

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# Install required packages if not already installed
!pip install -q dspy-ai torch

/opt/homebrew/bin/bash: line 1: pip: command not found


## Setup and Configuration

First, we'll set up the notebook with the required imports and configure DSPy with an appropriate LLM.

In [ ]:
#| export
import os
import json
import pickle
import dspy
from dspy.teleprompt import BootstrapFewShot, MIPROv2
from pathlib import Path
from unittest.mock import MagicMock

from cogitarelink_dspy.wrappers import get_tool_by_name
from cogitarelink_dspy.memory import ReflectionStore
from cogitarelink.core.graph import GraphManager

In [ ]:
# Configure DSPy with your preferred LLM
# You can use OpenAI, Anthropic, or any local model supported by DSPy

# Example for OpenAI
# If using OpenAI, set your API key in the environment or pass it directly
# os.environ["OPENAI_API_KEY"] = "your-api-key"
# lm = dspy.OpenAI(model="gpt-4o")

# Example for Anthropic
# os.environ["ANTHROPIC_API_KEY"] = "your-api-key"
# lm = dspy.LM("anthropic/claude-3-opus-20240229")

# For testing without an actual LLM, we'll use a mock LLM
class MockLM(dspy.LM):
    def __init__(self):
        self.history = []
    
    def basic_request(self, prompt, **kwargs):
        self.history.append(prompt)
        return ["This is a mock response for testing"]

lm = MockLM()
dspy.configure(lm=lm)

## Create or Load Training Data

Next, we'll load our development set for training the memory components. In this case, we'll use the examples from `devset_memory.jsonl`.

In [ ]:
#| export
def load_devset(path="../tests/devset_memory.jsonl"):
    """Load the memory development set from JSONL."""
    examples = []
    
    with open(path, 'r') as f:
        for line in f:
            data = json.loads(line)
            # Convert to DSPy Example format
            example = dspy.Example(
                q=data["q"],
                exp_tool=data["exp_tool"],
                use_memory=data.get("use_memory", False)
            ).with_inputs("q")
            examples.append(example)
            
    return examples

# Load the development set
devset = load_devset()

In [ ]:
# Display the development set
for i, example in enumerate(devset):
    print(f"Example {i+1}:")
    print(f"Query: {example.q}")
    print(f"Expected Tool: {example.exp_tool}")
    print(f"Use Memory: {example.use_memory}")
    print("---")

Example 1:
Query: Remember that wdt:P1476 is title
Expected Tool: AddReflection
Use Memory: False
---
Example 2:
Query: What's the Wikidata title property?
Expected Tool: RecallReflection
Use Memory: True
---
Example 3:
Query: Inject notes into system prompt
Expected Tool: ReflectionPrompt
Use Memory: False
---
Example 4:
Query: Store the fact that schema:name is a common property
Expected Tool: AddReflection
Use Memory: False
---
Example 5:
Query: Can you save rdfs:label for later reference?
Expected Tool: AddReflection
Use Memory: False
---
Example 6:
Query: Make a note that owl:sameAs indicates identity between resources
Expected Tool: AddReflection
Use Memory: False
---
Example 7:
Query: Remember that foaf:Person is a class for people
Expected Tool: AddReflection
Use Memory: False
---
Example 8:
Query: Remember that dc:creator represents the author
Expected Tool: AddReflection
Use Memory: False
---
Example 9:
Query: Store information that skos:broader indicates hierarchy
Expected T

## Define the Metric Function

We'll define a metric function to evaluate whether the agent is selecting the correct tool.

In [ ]:
#| export
def tool_match(pred, sample):
    """Check if the expected tool is in the trace."""
    return sample["exp_tool"] in pred.get("trace", [])

## Create a Memory Planner Agent

Now, we'll define a DSPy Module that integrates the memory tools.

In [ ]:
#| export
class MemoryPlanner(dspy.Module):
    """A DSPy module that selects the appropriate memory operation based on the query."""
    
    def __init__(self, graph_manager=None):
        super().__init__()
        
        # If no graph manager is provided, create a mock one for testing
        if graph_manager is None:
            graph_manager = MagicMock(spec=GraphManager)
        
        # Create the reflection store
        self.reflection_store = ReflectionStore(graph_manager)
        
        # Get all memory tools
        self.add_reflection = get_tool_by_name("AddReflection")()
        self.recall_reflection = get_tool_by_name("RecallReflection")()
        self.reflection_prompt = get_tool_by_name("ReflectionPrompt")()
        
        # Define the Chain of Thought for deciding which tool to use
        self.decide_tool = dspy.ChainOfThought("query -> decision: str")
        
    def forward(self, q):
        """Process a query and decide which memory tool to use."""
        trace = []
        
        # Use ChainOfThought to decide which tool to use
        tool_decision = self.decide_tool(query=q)
        
        # Based on the decision, choose the appropriate tool
        if "add" in tool_decision.decision.lower() or "store" in tool_decision.decision.lower() or "remember" in tool_decision.decision.lower():
            note_id = self.add_reflection(text=q, tags=["user_query"])
            trace.append("AddReflection")
            response = f"I've stored that information with ID: {note_id}"
            
        elif "recall" in tool_decision.decision.lower() or "retrieve" in tool_decision.decision.lower() or "what" in tool_decision.decision.lower():
            notes = self.recall_reflection(limit=3)
            trace.append("RecallReflection")
            if notes:
                notes_text = "\n".join([f"• {note.content['text']}" for note in notes])
                response = f"Here's what I recall:\n{notes_text}"
            else:
                response = "I don't have any relevant memories about that."
                
        elif "prompt" in tool_decision.decision.lower() or "format" in tool_decision.decision.lower() or "inject" in tool_decision.decision.lower():
            formatted = self.reflection_prompt(limit=5)
            trace.append("ReflectionPrompt")
            response = f"I've prepared these notes for inclusion in the prompt:\n{formatted}"
            
        else:
            response = "I'm not sure how to process that as a memory operation."
            
        return {"response": response, "trace": trace, "tool_decision": tool_decision.decision}

## Train the Memory Planner with DSPy

Now we'll train our memory planner using the DSPy BootstrapFewShot optimizer.

In [ ]:
#| export
def train_memory_planner(devset, metric=tool_match, num_iterations=3, graph_manager=None):
    """Train the memory planner using DSPy's compilation framework."""
    # Create the base planner
    planner = MemoryPlanner(graph_manager)
    
    # Set up the bootstrap trainer
    trainer = BootstrapFewShot(trainset=devset, metric=metric)
    
    # Configure search space for optimization
    search_space = {
        "RecallReflection.limit": [3, 5, 10],
        "ReflectionPrompt.limit": [3, 5, 10]
    }
    
    # When using a real LLM, uncomment this line to run compilation
    # optimized_planner = dspy.compile(planner, trainer, num_iterations=num_iterations, search_space=search_space)
    
    # For testing without a real LLM, we'll just return the unoptimized planner
    optimized_planner = planner
    
    return optimized_planner

In [ ]:
#| export
def train_memory_planner_simba(
    trainset,
    metric=tool_match,
    graph_manager=None,
    max_steps: int = 20,
    max_demos: int = 5,
    seed: int = 42,
):
    """Train the MemoryPlanner’s tool-selection policy using SIMBA."""
    planner = MemoryPlanner(graph_manager)
    import dspy
    simba = dspy.SIMBA(
        metric=metric,
        max_steps=max_steps,
        max_demos=max_demos,
    )
    optimized = simba.compile(
        student=planner,
        trainset=trainset,
        seed=seed,
    )
    return optimized


In [ ]:
# Train the memory planner
from cogitarelink.core.graph import GraphManager
graph_manager = GraphManager()
if len(devset) < 32:
    # fallback to BootstrapFewShot when dataset is too small
    print(f"Trainset too small ({len(devset)} < 32); using BootstrapFewShot instead")
    optimized_planner = train_memory_planner(devset, metric=tool_match, graph_manager=graph_manager)
else:
    optimized_planner = train_memory_planner_simba(
        trainset=devset,
        graph_manager=graph_manager,
        max_steps=20,
        max_demos=5,
        seed=42,
    )
print("✅ Training complete.")


2025/05/13 18:42:47 INFO dspy.teleprompt.simba: Starting batch 1 of 20.


AttributeError: 'MockLM' object has no attribute 'kwargs'

## Save and Load the Optimized Planner

Now we'll save the optimized planner so it can be distributed with the package.

In [ ]:
#| export
def save_optimized_planner(planner, path="../cogitarelink_dspy/optimized/memory_planner.pkl"):
    """Save the optimized memory planner to disk."""
    # Ensure the directory exists
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Save the planner using pickle
    with open(path, 'wb') as f:
        pickle.dump(planner, f)
    
    return path

def load_optimized_planner(path="../cogitarelink_dspy/optimized/memory_planner.pkl", graph_manager=None):
    """Load the optimized memory planner from disk."""
    # Check if the file exists
    if not os.path.exists(path):
        raise FileNotFoundError(f"Optimized planner not found at {path}")
    
    # Load the planner using pickle
    with open(path, 'rb') as f:
        planner = pickle.load(f)
    
    # If a graph manager is provided, update the planner's reflection store
    if graph_manager is not None:
        planner.reflection_store = ReflectionStore(graph_manager)
    
    return planner

In [ ]:
# Save the optimized planner
saved_path = save_optimized_planner(optimized_planner)
print(f"Saved optimized planner to {saved_path}")

## Test the Optimized Planner

Let's test the optimized planner with some examples.

In [ ]:
# Load the optimized planner
loaded_planner = load_optimized_planner()

# Test with a few examples
test_queries = [
    "Remember that owl:sameAs is used for identity statements",
    "What do you know about Wikidata properties?",
    "Format the recent notes for inclusion in the prompt"
]

for query in test_queries:
    result = loaded_planner(q=query)
    print(f"Query: {query}")
    print(f"Selected Tool: {result['trace']}")
    print(f"Response: {result['response']}")
    print("---")

## Integration with the Cogitarelink DSPy Agent

Now we'll show how to integrate the optimized memory planner with the main Cogitarelink DSPy agent.

In [ ]:
#| export
class CogitarelinkAgent(dspy.Module):
    """A DSPy agent for Cogitarelink with integrated reflection memory."""
    
    def __init__(self, graph_manager=None):
        super().__init__()
        
        # If no graph manager is provided, create a mock one for testing
        if graph_manager is None:
            graph_manager = MagicMock(spec=GraphManager)
        
        # Load the optimized memory planner
        try:
            self.memory_planner = load_optimized_planner(graph_manager=graph_manager)
        except FileNotFoundError:
            # Fall back to unoptimized planner if optimized version not found
            self.memory_planner = MemoryPlanner(graph_manager)
        
        # Main agent's Chain of Thought
        self.main_cot = dspy.ChainOfThought("query, context -> response")
        
        # Tool selection module
        self.select_tool = dspy.ChainOfThought("query -> tool_name: str, is_memory: bool")
        
    def forward(self, query):
        """Process a query using the appropriate tools and memory."""
        # Decide whether to use memory tools or other tools
        tool_selection = self.select_tool(query=query)
        
        if tool_selection.is_memory:
            # Use the memory planner for memory operations
            memory_result = self.memory_planner(q=query)
            return {
                "response": memory_result["response"],
                "tool_used": memory_result["trace"][0] if memory_result["trace"] else "None",
                "is_memory_operation": True
            }
        else:
            # For non-memory operations, use other tools...
            # Get recent memories as context
            context = self.memory_planner.reflection_prompt(limit=3)
            
            # Use main Chain of Thought with memory context
            result = self.main_cot(query=query, context=context)
            
            # Optionally store the interaction in memory
            if "important information" in query.lower():
                self.memory_planner.add_reflection(
                    text=f"User asked: {query} | Response: {result.response}",
                    tags=["interaction"]
                )
            
            return {
                "response": result.response,
                "tool_used": tool_selection.tool_name,
                "is_memory_operation": False
            }

In [ ]:
# Create the full agent
agent = CogitarelinkAgent()

# Test with different types of queries
queries = [
    "Remember that rdfs:label is used for display names",  # Memory operation
    "What is the capital of France?",  # Regular query
    "What important information do you have about Wikidata properties?"  # Query that uses memory context
]

for query in queries:
    result = agent(query=query)
    print(f"Query: {query}")
    print(f"Tool Used: {result['tool_used']}")
    print(f"Is Memory Operation: {result['is_memory_operation']}")
    print(f"Response: {result['response']}")
    print("---")

## Conclusion

In this notebook, we've demonstrated how to:

1. Define a memory planner that integrates with DSPy
2. Train and optimize the planner using DSPy's compilation framework
3. Save and load the optimized planner for distribution
4. Integrate the memory system with a full Cogitarelink agent

This approach allows for declarative optimization of the memory system and ensures that agents can effectively utilize semantic memory for reflective learning.

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()